In [6]:
import pandas as pd
from tqdm.notebook import tqdm_notebook

In [7]:
data = pd.read_csv('data/headlines/1986.csv')


In [18]:
for i in tqdm_notebook(range(1987, 2022)):
    data = pd.concat([data, pd.read_csv('data/headlines/'+ str(i) +'.csv')])

data = data.reset_index(drop = True)

  0%|          | 0/35 [00:00<?, ?it/s]

In [20]:
data.shape

(9919470, 7)

In [21]:
data.head()

,Unnamed: 0,headline,date,doc_type,material_type,section,keywords
0,0,CAROLINA DEMOCRATS SEE '86 BOON IN REAGAN VETO,1986-01-02,article,News,U.S.,"['LABOR', 'UNEMPLOYMENT AND JOB MARKET', 'Inte..."
1,1,EVENTS: CARPETS AS ART,1986-01-02,article,News,Home & Garden,['Entertainment and Amusements']
2,2,Percy on Georgetown,1986-01-02,article,News,U.S.,"['LITTERING', 'STREETS', 'Noise', 'Waste Mater..."
3,3,APARTMENT OF THE MOMENT: THE LATEST IN TRENDS,1986-01-02,article,News,Home & Garden,"['INTERIOR DECORATION', 'APARTMENT HOUSES']"
4,4,INVESTORS OPEN '86 CONFIDENTLY,1986-01-02,article,News,Business Day,"['NINETEEN HUNDRED EIGHTY-FIVE', 'Stocks and B..."


In [26]:
relevant_headlines = data[(data['section'] == 'Business Day') | (data['section'] == 'World')]

In [28]:
sample = relevant_headlines.sample(3000)
sample = sample.iloc[:,1:3]
sample.head()

,headline,date
9130313,Answers to Readers’ Questions About State’s Se...,2010-11-29
3545478,Seagate Technology reports earnings for Qtr to...,1988-07-20
293687,Efforts by 2 Latin Presidents Appear to Break ...,1988-09-18
7372729,Hiram Walker is seeking to create a less 'fudd...,1994-09-08
5676355,Flowers and Fubon Said to Eye A.I.G.’s Taiwan ...,2009-06-19


In [31]:
sample.to_csv('data/headlines/for_labeling.csv')

# Some manual labeling happened here...

In [113]:
labeled = pd.read_csv('data/headlines/labeled1.csv')

In [114]:
labeled['headline'] = labeled['headline'].str.lower()
labeled["headline"] = labeled['headline'].str.replace('[^\w\s]','')
labeled['headline'] = labeled['headline'].str.strip()
labeled['headline'] = labeled['headline'].str.split()

<ipython-input-114-4c80adde6d6d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labeled["headline"] = labeled['headline'].str.replace('[^\w\s]','')


In [115]:
train_data = labeled

print(labeled['isgood'].value_counts() / labeled.shape[0] * 100)

print(train_data['isgood'].value_counts() / train_data.shape[0] * 100)


print(test_data['isgood'].value_counts() / test_data.shape[0] * 100)


0    70.4
1    29.6
Name: isgood, dtype: float64
0    70.4
1    29.6
Name: isgood, dtype: float64
0    70.5
1    29.5
Name: isgood, dtype: float64


In [116]:
vocabulary = list(set(train_data['headline'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [117]:
Pspam = train_data['isgood'].value_counts()[1]/train_data.shape[0]

In [118]:
Pham = 1-Pspam

In [119]:
Nspam = train_data[train_data['isgood'] == 1].apply(len).sum()

In [120]:
Nham = train_data[train_data['isgood'] == 0].apply(len).sum()


In [121]:
Nvoc = len(train_data.columns) - 3
alpha = 1

In [122]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['isgood'] == 1, word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1
      
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['isgood'] == 0, word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

In [123]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)

    print('P_spam', p_spam_given_message)
    print('P_ham', p_ham_given_message)



    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [124]:
test = test_data.loc[45]['headline']

In [125]:
classify(test)

P_spam 2.273298375734536e-43
P_ham 2.5142624313395992e-46


'spam'

In [127]:
for i in relevant_headlines['headline']:
    print(i)

’
Spared for Months, Vietnam Faces a Wave of New Infections
Without humans around, Sri Lanka’s zoo animals enjoy a pandemic baby boom.
Etsy is buying the fashion resale app Depop for $1.6 billion.
Archaeologists Uncover Decapitated Bodies From Roman Britain
How ViacomCBS Saved Billions in Taxes
Russian Cybercriminal Group Was Behind Meat Plant Attack, F.B.I. Says
Jobs lost to the pandemic will take years to return, a labor organization says.
Surveying Goma’s Volcanoes
Anheuser-Busch to give away free beer when the U.S. hits Biden’s July 4 vaccination goal.
Would a new government mean a profound shift for Israel?
The U.S. imposes — and suspends — tariffs on six countries over digital taxes.
A See-Through Pool Opens 10 Stories Above London
The Arab party Raam makes history within coalition.
‘Complete Insanity’: Virus-Stricken Brazil Agrees to Host Soccer Tournament
Hitler’s ‘Mein Kampf’ Gets New French Edition, With Each Lie Annotated
After Eruption, Residents in Congo Struggle to Find F